<a href="https://colab.research.google.com/github/ugrasegehan/IE423/blob/main/Task8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IE423
Task8

## Egehan Uğraş
22003278

# Joke Recommender

## Initialize

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Joke metadata
dfJokes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/task8/JokeText.csv')

# User ratings for each joke
dfRtg1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/task8/UserRatings1.csv')
dfRtg2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/task8/UserRatings2.csv')

dfRating = pd.concat([dfRtg1, dfRtg2], axis=1)

In [4]:
dfRating

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User73412,User73413,User73414,User73415,User73416,User73417,User73418,User73419,User73420,User73421
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,3.64,4.32,6.99,-9.66,-8.4,-0.63,9.51,-7.67,-1.6,8.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,6.31,-1.02,3.98,3.93,9.13,1.94,0.44,1.21,6.94,...,2.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,96,-4.95,-0.97,-6.46,-2.57,9.17,1.99,-0.78,5.34,5.83,...,NaN,NaN,NaN,NaN,NaN,0.78,NaN,NaN,NaN,NaN
97,97,-0.19,4.13,-6.89,1.07,9.17,3.45,-1.02,1.94,5.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,98,3.25,-1.84,-2.33,2.33,9.08,9.17,1.70,3.06,6.55,...,7.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Building Recommendations

Let us first try to build a recommender using movie content such as descriptions and taglines, also known as **Content Based Filtering**

### 1. Content Based Filtering

The idea here is to determine how similar the descriptions are based on the terms used in the descriptions - while ignoring commonly used words.  Then recommend other items with similar descriptions.  In order to do this, **TF-IDF Vectorization** is used.

#### Prepare data

In [5]:
dfJokes.head()

,JokeId,JokeText
0,0,"A man visits the doctor. The doctor says ""I ha..."
1,1,This couple had an excellent relationship goin...
2,2,Q. What's 200 feet long and has 4 teeth? \n\nA...
3,3,Q. What's the difference between a man and a t...
4,4,Q.\tWhat's O. J. Simpson's Internet address? \...


In [6]:
dfJokes.shape

(100, 2)

In [7]:
# Remove duplicates
dfJokes.drop_duplicates(subset ='JokeText', keep = 'first', inplace = True)
dfJokes.shape

(100, 2)

So no duplication is observed, then we do not need an extra column like description in n8

#### Build Model

In [8]:
# Generate a matrix of common terms that show up in each movie

from sklearn.feature_extraction.text import TfidfVectorizer
mdlTfvJokes = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=1, stop_words='english')
tfidf_matrix = mdlTfvJokes.fit_transform(dfJokes['JokeText'])
tfidf_matrix.shape

(100, 3774)

The similarity between any two movies (x) and (y) is defined as the **Cosine Similarity**:
cosine(x,y)=x.y⊺||x||.||y||

Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score.

In [9]:
# Calculate cosine similarity between each pair of movies as a function of the similarity of the common terms

from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(100, 100)

#### Predict

In [10]:
# Prepare recommendation function (build code from scratch and then package as function for ease of understanding)
dfJokes['JokeId'] = dfJokes['JokeId'].astype(str)
titles = dfJokes['JokeId']
indices = pd.Series(dfJokes.index, index=dfJokes['JokeId'])

def get_recommendations(JokeText):
    idx = indices[JokeText]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

Let's get 10 joke recommendations for joke 15,

In [11]:
get_recommendations('15').head(10)

97    97
21    21
36    36
8      8
13    13
61    61
0      0
1      1
2      2
3      3
Name: JokeId, dtype: object

15 joke recommendations for Joke 19

In [12]:
get_recommendations('19').head(15)

77    77
72    72
29    29
3      3
20    20
74    74
91    91
99    99
0      0
38    38
83    83
11    11
27    27
8      8
67    67
Name: JokeId, dtype: object

These recommendations focus on jokes that are similar in content and theme. When someone queries our engine for joke recommendations based on a particular joke, they will receive the same suggestions for that joke, regardless of who they are.

However, what if we also had information about individual humor preferences? Could we tailor the recommendations to better align with each user's unique taste? For this, we can employ a technique called Collaborative Filtering. This method is based on the concept that users with similar humor preferences to mine can help predict which jokes I might find funny, even if I haven't heard them yet. By leveraging the experiences of similar users, we can provide more personalized and relevant joke recommendations.

utexas_ds_orie_divider_gray.png

### 2. Collaborative Filtering

The idea here is that a user x joke matrix is decomposed into a product of user x concept, concept x concept, and concept x joke matrices. These can then be used to derive similarities between users' humor preferences. This process is known as Singular Value Decomposition and is applied in the context of a joke recommendation system.

#### Prepare data

In [13]:
dfRating.head(10)

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User73412,User73413,User73414,User73415,User73416,User73417,User73418,User73419,User73420,User73421
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,3.64,4.32,6.99,-9.66,-8.40,-0.63,9.51,-7.67,-1.60,8.30
5,5,1.75,-0.78,1.26,6.65,8.25,-8.11,-6.75,2.14,0.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,4.76,1.60,-5.39,-7.52,4.08,4.42,-0.15,-0.24,-3.01,...,4.56,6.21,6.99,5.15,4.37,-3.64,-4.95,NaN,-5.29,-1.36
7,7,3.30,1.07,1.50,7.28,2.52,2.72,-5.87,8.06,-6.65,...,0.87,-6.36,6.70,-9.37,-3.74,-7.23,8.59,-8.79,-3.69,0.29
8,8,-2.57,-8.69,-8.40,-5.15,-9.66,9.08,-3.54,2.82,-3.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,-1.41,-4.66,4.37,-7.14,2.48,9.13,-5.19,7.52,1.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
dfRating.columns

Index(['JokeId', 'User1', 'User2', 'User3', 'User4', 'User5', 'User6', 'User7',
       'User8', 'User9',
       ...
       'User73412', 'User73413', 'User73414', 'User73415', 'User73416',
       'User73417', 'User73418', 'User73419', 'User73420', 'User73421'],
      dtype='object', length=73423)

#### Build Model

In [15]:
# Prepare data into Surprise library format

!pip3 install scikit-surprise #or !conda install -c conda-forge scikit-surprise
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

dfNRating = dfRating.transpose()
dfNRating = dfNRating.drop(dfNRating.index[0])
dfNRating.columns = [f'JokeId{i-1}' for i in range(1, dfNRating.shape[1] + 1)]
dfNRating.reset_index(drop=True, inplace=True)
dfNRating.index += 1
dfNRating['UserId'] = range(1, len(dfNRating) + 1)
dfNRating = dfNRating[['UserId'] + dfNRating.columns[:-1].tolist()]
user_ids_in_dfRating = [int(col.split('User')[1]) for col in dfRating.columns if 'User' in col]
dfNRating['UserId'] = user_ids_in_dfRating
dfNRating.head(33000)



reader = Reader(rating_scale=(0,5))
df_melted = dfNRating.melt(id_vars=['UserId'], var_name='JokeId', value_name='Rating')

X = Dataset.load_from_df(df_melted[['UserId', 'JokeId', 'Rating']], reader)
X_train, X_test = train_test_split(X, test_size=.2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357278 sha256=a08e25833bbaf84f0ba207990ba64e4041bdbf752c82125de415b38adc554166
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [16]:
# Define SVD model

from surprise import SVD

mdlSvdJokesRtg = SVD()

In [17]:
# Fit SVD model

mdlSvdJokesRtg.fit(X_train)
test_pred = mdlSvdJokesRtg.test(X_test)

In [18]:
# Evalute SVD accuracy

from surprise import accuracy

accuracy.rmse(test_pred)

RMSE: nan


nan

Hence RMSE did not give any results, let me check missing values.

In [19]:

# Check for missing values in the ratings
print(df_melted.isnull().sum())

# If there are missing values, fill them with a reasonable value (e.g., the average rating)
df_melted['Rating'].fillna(df_melted['Rating'].mean(), inplace=True)

# Recreate the Surprise Dataset and retrain the model
X = Dataset.load_from_df(df_melted[['UserId', 'JokeId', 'Rating']], reader)
X_train, X_test = train_test_split(X, test_size=.2)

mdlSvdJokesRtg = SVD()
mdlSvdJokesRtg.fit(X_train)

test_pred = mdlSvdJokesRtg.test(X_test)
print(accuracy.rmse(test_pred))


UserId          0
JokeId          0
Rating    3205740
dtype: int64
RMSE: 3.7412
3.7411700641972665


In [ ]:
# Tune hyperparameters

from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10, 15], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(X)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

nan
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}


In [20]:
# Cross-validate

from surprise.model_selection import cross_validate

cross_validate(mdlSvdJokesRtg, X, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.7502  3.7408  3.7338  3.7362  3.7356  3.7393  0.0059  
MAE (testset)     2.6074  2.5982  2.5951  2.5958  2.5942  2.5981  0.0048  
Fit time          158.30  168.78  163.66  170.04  165.45  165.25  4.15    
Test time         21.09   25.72   21.46   25.85   21.87   23.20   2.13    


{'test_rmse': array([3.75021725, 3.7407559 , 3.73377562, 3.73621829, 3.73561993]),
 'test_mae': array([2.60739513, 2.5982054 , 2.59514779, 2.59583418, 2.59415017]),
 'fit_time': (158.3004767894745,
  168.77970790863037,
  163.66297149658203,
  170.03639197349548,
  165.4536051750183),
 'test_time': (21.093117237091064,
  25.72137713432312,
  21.46452522277832,
  25.8547785282135,
  21.87333655357361)}

Let us now use the trained model to arrive at predictions.

#### Predict

Let's first see which jokes user # 1 has already viewed.

In [21]:
dfJokes[dfJokes['JokeId'] == 1]

,JokeId,JokeText


Now, let's predict what rating user # 1 would give to joke # 32 (since he/she hasn't seen it yet)

In [23]:
mdlSvdJokesRtg.predict(1, 32)

Prediction(uid=1, iid=32, r_ui=None, est=1.1767442118733422, details={'was_impossible': False})

I could not understand why it did not work, most probably it might be from lack of tuning hence the RAM was not available to do so.

## Takeaways

* Introduced content-based filtering to recommend items based on their descriptions using *TF-IDF Vectorization*
* In the event that user preference data is available, collaborative filtering is leveraged to recommend items based on other similar users using *Singular Value Decomposition*

<< WHITEBOARD

utexas_ds_orie_divider_gray.png